In [1]:
using LightGraphs

"""
inverse logit function
"""
invLogit(x) = 1./(1.+e.^-x)   

"""
given graph and probability, adds a node which must have >0
connections by flipping biased coin for each existing node
"""
function addNode2(graph, p)
    add_vertex!(graph)
    x = nv(graph)
    degree = 0
    while degree ==0
        flips = rand(x-1)
        for i = 1:x-1
            if p[i]>flips[i]
                add_edge!(graph,i,x)
                degree +=1 
            end
        end
    end
    return graph
end


"""
given graph, b vector, and a_0, adds a new node as specifiec by the model
"""
function addPrefNode(g,b,a_0 = -7)
    n = nv(g)
    L = laplacian_matrix(g)
    a = lufact(L) \ (b - mean(b))    
    p = invLogit(a+a_0)
    addNode2(g,p)
    push!(b,0)
    return g
end


"""
given graph and number of new edges desired, randomly adds edges between existing nodes
"""
function randEdgeGen(graph, newedges)
    for i in 1:newedges
        z = newedges
        x = collect(1:nv(graph))
        edge1 = rand(x)
        deleteat!(x, edge1)
        edge2 = rand(x)
        add_edge!(graph,edge1,edge2)
    end
    return graph
end
;

In [2]:
"""
soft threshold
"""
soft(c,lambda) = sign(c).*max(abs(c)-lambda/2,0)

"""
computes gradient
"""
function gradient2(a,a_0,u,L,rho,b,y)
    grad = -1.*(y-invLogit(a+a_0))+L*u + rho*L*(L*a-b)
    return grad
end;


"""
computes hessian
"""
function hessian(a,a_0,rho,L)
    hess = Diagonal(vec((invLogit(a+a_0).*(1-invLogit(a+a_0)))))+rho*L^2
    return hess
end;




"""
newton raphson for a update
"""
function newton(y_i,a_0,L,rho,b,u)
    a = zeros(length(y_i),1)
    a_old = a
    iters = 0
    diff = 1.0
    while(diff >STOP_DIFF && iters< MAX_ITER )
        grad = gradient2(a_old,a_0,u,L,rho,b,y_i)
        hess = hessian(a_old,a_0, rho,L)
        a = a_old - inv(hess)*grad
        diff = norm(a-a_old)
        a_old = a
        iters = iters+1
    end
    if(iters == MAX_ITER)
        print("max iter reached")
    end
    return a
end
;




One new node

In [3]:
levels = 10     #number of levels in binary tree
g = BinaryTree(levels)
n = nv(g)
b = (rand(n) .< 8 / n)*1. 
genb = copy(b)  # save for later
g = randEdgeGen(g,1000)
L =  laplacian_matrix(g)
numnewnodes = 1
a_0 = -5
g = addPrefNode(g,b, a_0)
connects = zeros(2^levels-1,1)  #-1 for -1 1 coding
connects[neighbors(g,nv(g))] = 1
A = connects


const MAX_ITER = 500
const STOP_DIFF = 0.001;


t = 2^levels-1+numnewnodes #number of nodes at time t
t_0 = 2^levels-1  # number of initial nodes
;


rho = 1.5
lambda = 1.1
new = numnewnodes
a = zeros(t-1,new)
b = zeros(t_0)
u = zeros(t-1,new)
#	alpha = 1.5  #relaxation parameter
iters = 0
diff = 1.0
b_old = b;

In [13]:
for j in 1:200     
    a= newton(A,a_0,L,rho,b,u)
    #b update
    c = zeros(t-1)
    c = c+ u+rho*(L*a)/(rho)
    b = soft(c,lambda)
    u = u+ rho*(L*a-b)
    diff  = norm(b-b_old)
    b_old = b
    println(diff)
end

0.000836347873060465
0.0008354316214928881
0.0006672526816735149
0.0006553576766229632
0.0006511611413483926
0.0006470233816955759
0.0006429018357399845
0.0006389534991939756
0.00063531712391909
0.0006321019938613273
0.0006293800431859508
0.0006271814218820053
0.000625493877808813
0.0006242659030991608
0.0006234131701084693
0.0006228274398375794
0.0006223869218479585
0.0006219670185287581
0.0006214504794865562
0.0006207361792966513
0.000619745962513696
0.0006184292303119802
0.000616765146422235
0.000614762505957313
0.0006124574415099724
0.0006099092454179902
0.000607194674690317
0.0006044011815803001
0.0006016195777858801
0.0005989366866444024
0.0005964285533499017
0.0005941547546882543
0.0005921542674486691
0.000590443217798864
0.0005890146537896119
0.0005878402823136681
0.0005868739195204687
0.0005860562482370621
0.0005853203778527266
0.0005845976693687215
0.0005838233161452828
0.0005829412453238255
0.0005819080077825333
0.0005806954385473551
0.0005792919817181725
0.00057770267620479

In [14]:
sort(b[:,1])

1023-element Array{Float64,1}:
 -0.541204 
 -0.285431 
 -0.251063 
 -0.229683 
 -0.216996 
 -0.216996 
 -0.159308 
 -0.112286 
 -0.0942326
 -0.0925654
 -0.0624468
 -0.0624312
 -0.0605712
  ⋮        
 -0.0      
 -0.0      
 -0.0      
 -0.0      
 -0.0      
 -0.0      
 -0.0      
 -0.0      
 -0.0      
 -0.0      
 -0.0      
  2.64821  